In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import glob

In [2]:
#select DAYS and RATS - depending on how your files are saved
#this is done for filtering paths
rats=['\\5\\']
days=['2023-04-04']
#mark path where your data is
data_folder=r"C:\test*.csv"
#read all paths
paths = glob.glob(data_folder)

In [3]:
#list for saving results
res_movement=[]
#for every rat id
for rat in range(0,len(rats)):
    #every day
    for day in range(0,len(days)):    
        #selects the appropriate path
        path=list(filter(lambda x: (days[day] in x) and (rats[rat] in x), paths))
        #if the path of selected day and rat id exists
        if len(path)>0:
            #select the first one in case multiple are found
            path=path[0]
            #read data to Python
            data=pd.read_csv(path,header=None, names=['movement','time'])
            data=data.set_index(pd.to_datetime(data['time']))
            data['time']=pd.to_datetime(data['time'])
            data['movement_group']=(data.movement.ne(data.movement.shift())).cumsum()
            if len(data)>0:
                #integrate over 60 min - can be changed
                hours=data.index.round('60min').drop_duplicates()
                times_moving=[]
                for i in range(0, len(hours)-1):
                    df=data[hours[i]:hours[i+1]]
                    df=df[df.movement]
                    if len(df)>0:
                        s = df.reset_index(drop=True).groupby('movement_group')['time'].apply(lambda x: x.iloc[-1] - x.iloc[0])
                        times_moving.append(s.sum().total_seconds())
                    else:
                        times_moving.append(0)
                one_rat_one_day=pd.DataFrame(times_moving, columns=['time_moving'])
                one_rat_one_day['id']=rats[rat]
                one_rat_one_day['day']=data.index.round('60min').drop_duplicates().strftime('%Y-%m-%d').tolist()[0:-1]
                one_rat_one_day['hour']=data.index.round('60min').drop_duplicates().strftime('%H').tolist()[0:-1]
                res_movement.append(one_rat_one_day)
res_movement=pd.concat(res_movement).reset_index(drop=True)